In [3]:
# 初回だけ実行
# %pip install rp2040-pio-emulator@git+https://github.com/NathanY3G/rp2040-pio-emulator
# %pip install adafruit-circuitpython-pioasm

In [2]:
from adafruit_pioasm import assemble
from pioemu import emulate

program = """
.program quickstart_example
    set x 9
loop:
    jmp x-- loop
    wait 1 gpio 0
"""


generator = emulate(assemble(program), stop_when=lambda _, state: state.x_register < 0)

for before, after in generator:
  print(f"[{after.clock}] X register: {before.x_register} -> {after.x_register}")

[1] X register: 0 -> 9
[2] X register: 9 -> 8
[3] X register: 8 -> 7
[4] X register: 7 -> 6
[5] X register: 6 -> 5
[6] X register: 5 -> 4
[7] X register: 4 -> 3
[8] X register: 3 -> 2
[9] X register: 2 -> 1
[10] X register: 1 -> 0
[11] X register: 0 -> -1
